# Demo of `columnsfmri` 
## Simulation and optimization of fMRI of cortical columns

Import model implementation from columnsfmri.py and other useful modules.

In [ ]:
import columnsfmri

%matplotlib inline
import numpy as np
import importlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

### A tour of the inner workings of the model

Inititialize simulation using a 512 x 512 grid on an area of 24 x 24 mm. 

In [ ]:
N = 512; L = 24
sim = columnsfmri.simulation(N,L)

Simulate a column pattern by filtering Gaussian white noise. Rho is the main pattern frequency, delta specifies the amount of irregularity.

In [ ]:
gwn = sim.gwnoise()
rho,deltaRelative = 0.2,1
columnPattern = sim.columnPattern(rho,deltaRelative,gwn)
sim.plotPattern(columnPattern)

Simulate a spatial BOLD response with a FWHM of 2 mm, and an average response amplitude of 5%.

In [ ]:
fwhm = 2
beta = 0.05
boldPattern,_,_ = sim.bold(fwhm,beta,columnPattern)
sim.plotPattern(boldPattern)

Simulate MRI sampling using a voxel width of 3 mm. (We first add the relative response pattern to a constant background of 1).

In [ ]:
w = 1
mriPattern = sim.mri(w,1+boldPattern)
sim.plotVoxels(mriPattern)

The amount of functional contrast _c_ can be quantified as the standard deviation of the imaged responses (contrast range).

In [ ]:
c = np.std(mriPattern)
print('c = %.2f%%' % (100*c))

Simulate the noise level as a function of voxel width.

In [ ]:
w = np.linspace(0.1,3,100)
V = w**3
TR = 2
nT = 1
differentialFlag = True
noiseType = '3T'
SNR = 1/columnsfmri.noiseModel(V,TR,nT,differentialFlag,noiseType=noiseType)

plt.plot(w,SNR)
plt.xlabel('voxel width [mm]')
plt.ylabel('multi measurement SNR')
plt.title('3T, TR = 2s, nT = 100')
plt.show()

SNR for a voxel width of 2:

In [ ]:
w = 2
V = w**3
SNR = 1/columnsfmri.noiseModel(V,TR,nT,differentialFlag,noiseType=noiseType)
print('SNR = %.2f' % SNR)

Contrast to noise ratio = c * SNR:

In [ ]:
CNR = c * SNR
print('CNR = %.2f' % CNR)

Calculate detection probability from CNR and number of voxels.

In [ ]:
CNR = 1
nVoxels = 10
columnsfmri.detectionProbability(CNR,nVoxels)

Add noise to MRI pattern:

In [ ]:
 mriPlusNoisePattern = mriPattern + \
    (1/SNR) * np.random.randn(*mriPattern.shape)
sim.plotVoxels(mriPlusNoisePattern)

Calculate the correlation between the original and the (interpolated) imaged pattern.

In [ ]:
sim.patternCorrelation(columnPattern,mriPlusNoisePattern)

### Run optimization simulation

Set standard parameters for optimization simulation.

In [ ]:
parameters = columnsfmri.setParameters('irregular')
for parameter,value in parameters.items():
    print(parameter + ": " + str(value))

Run optimization simulation:

In [ ]:
results = columnsfmri.simulatefMRIOfColumnPatterns(parameters)

Summarize results:

In [ ]:
columnsfmri.printResults(results)